# Storing data with Spark
Documentation on storing data with spark in general can be found:
 http://spark.apache.org/docs/latest/sql-data-sources-load-save-functions.html

In Wix we use ParquetTableStorer wrapper.

In [2]:
import sys
sys.path.append('/home/iceberg/notebooks/PyCon_LT_Workshop')

from helpers.utils import get_spark_session, get_yellow_taxi_data, get_dim_data, ROOT_DIR
from pyspark.sql import functions as f
spark = get_spark_session("joins")

yellow_taxi_data = get_yellow_taxi_data(spark=spark)

dim_taxi_zones, dim_rates, dim_payments, dim_vendor = get_dim_data(spark)

24/02/17 09:07:34 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


We have two options now:

1. Overwrite the table completely
2. Append information (use it only with partitioned data!) otherwise results might not be what you expect

In [3]:
yellow_taxi_data.write.mode('overwrite').parquet(ROOT_DIR + "output/yellow_taxi.parquet")

In [4]:
(yellow_taxi_data
 .filter("fare_amount>100")
 .join(dim_vendor, yellow_taxi_data.VendorID == dim_vendor.vendor_id, "full")
 .write
 .mode('overwrite')
 .parquet(ROOT_DIR + "output/yellow_taxi_filtered.parquet"))